In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

def scale_score(raw_score, min_score=300, max_score=900):
    """Scales the raw score to fit within the specified range."""
    scaled = np.clip((raw_score - 0.5) * (max_score - min_score) + min_score, min_score, max_score)
    return int(scaled)

def preprocess_data(df):
    """Preprocesses the dataset by dropping irrelevant columns and handling missing values."""
    columns_to_drop = ['id', 'member_id', 'url', 'desc', 'title', 'zip_code', 
                       'issue_d', 'last_pymnt_d', 'next_pymnt_d', 
                       'last_credit_pull_d', 'pymnt_plan', 'emp_title']
    
    # Save the 'member_id' before dropping it
    member_ids = df['member_id']
    
    # Drop irrelevant columns
    df = df.drop(columns=columns_to_drop)
    imputer_num = SimpleImputer(strategy='mean')
    imputer_cat = SimpleImputer(strategy='most_frequent')
    
    # Impute missing values for numerical and categorical columns
    df[df.select_dtypes(include=['float64', 'int64']).columns] = imputer_num.fit_transform(df.select_dtypes(include=['float64', 'int64']))
    df[df.select_dtypes(include=['object']).columns] = imputer_cat.fit_transform(df.select_dtypes(include=['object']))
    
    return df, member_ids

def generate_scorecard(data_path, target_column='loan_status', test_size=0.2, random_state=42):
    """Generates a scorecard using logistic regression on the provided dataset."""
    # Load and preprocess data
    df = pd.read_csv(data_path, low_memory=False)
    df, member_ids = preprocess_data(df)

    # Split data into features and target variable
    X = pd.get_dummies(df.drop(columns=[target_column]))
    y = (df[target_column] == 'Charged Off').astype(int)

    # Scale features
    X_scaled = StandardScaler().fit_transform(X)

    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=test_size, random_state=random_state)

    # Define and fit logistic regression model
    model = LogisticRegression(max_iter=100, solver='liblinear')  # 'liblinear' is suitable for small datasets

    # Cross-validation for reliable AUC score
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=random_state)
    scores = cross_val_score(model, X_train, y_train, scoring='roc_auc', cv=cv)

    # Train model on full training set
    model.fit(X_train, y_train)

    # Predictions and evaluation
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    auc_score = roc_auc_score(y_test, y_pred_proba)
    y_pred = model.predict(X_test)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Scale AUC score
    scaled_auc = scale_score(auc_score)

    # Create scorecard DataFrame
    scorecard = pd.DataFrame({
        'Metric': ['Mean AUC', 'Scaled Score', 'Confusion Matrix'],
        'Score': [scores.mean(), scaled_auc, conf_matrix.tolist()]
    })

    # Generate predicted probabilities for the entire dataset
    all_predictions = model.predict_proba(X_scaled)[:, 1]

    # Map the predicted probabilities to member_ids and scale them
    member_scores = pd.DataFrame({
        'member_id': member_ids,  # Map the original member_ids
        'raw_score': all_predictions
    })
    # Apply scaling to the predicted probabilities for each member
    member_scores['scaled_score'] = member_scores['raw_score'].apply(scale_score)
    return scorecard, member_scores

# Run the scorecard generation and get results
scorecard, member_scores = generate_scorecard('loan_data_2015.csv')
# Print the scorecard and a preview of member-based scores
print(scorecard)
print(member_scores.head())  


             Metric                    Score
0          Mean AUC                 0.998708
1      Scaled Score                      596
2  Confusion Matrix  [[83660, 4], [33, 522]]
   member_id     raw_score  scaled_score
0   64537751  1.000000e+00           599
1   64163931  4.501982e-08           300
2   64333218  9.946566e-01           596
3   63900496  9.996503e-01           599
4   62544456  9.999506e-01           599
